In [1]:
import pandas as pd
import pickle
import seaborn as sns
import numpy as np
path = '../../../../../../Desktop/Orthografietrainer2020/pickle/'

Schueler

In [3]:
infile = open(path+'schueler2020only.pkl','rb')
schueler = pickle.load(infile)
infile.close()
schueler = schueler[['ID','Geschlecht','Klassenstufe','Anmeldeklassenstufe']]

In [4]:
schueler.rename(columns = {'ID':'UserID'}, inplace = True)
#Geschlecht: bereinigung
schueler['Geschlecht'] = schueler['Geschlecht'].replace(['m.','armin.zekan@hakamste'],'m')
schueler['Geschlecht'] = schueler['Geschlecht'].replace(['w.','we', 'alma.dautbegovic@hak', 'WO m'],'w')
schueler['Geschlecht'] = schueler['Geschlecht'].replace(['d','wm'],np.nan)

#Klassenstufe: nutze nur Klassenstufe 5-12
options = ['5', '6','7','8','9','10','11','12'] 
schueler = schueler[schueler['Klassenstufe'].isin(options)] 
schueler['Klassenstufe'] = schueler['Klassenstufe'].astype('int')

#Jahredabei: erstelle aus Klassenstufe - Anmeldeklassenstufe die Spalte Jahredabei
schueler = schueler[schueler['Anmeldeklassenstufe'].isin(options)] 
schueler['Anmeldeklassenstufe'] = schueler['Anmeldeklassenstufe'].astype('int')
schueler['Jahredabei'] = schueler['Klassenstufe'] - schueler['Anmeldeklassenstufe']

#Geschlecht Dummy
dummy_Geschlecht = pd.get_dummies(schueler['Geschlecht'], prefix='Sex_')
schueler = pd.merge(left=schueler, right=dummy_Geschlecht,left_index=True,right_index=True)

# Lösche 
schueler= schueler.drop(columns=['Anmeldeklassenstufe','Geschlecht'])

Sitzungssummary

In [5]:
infile = open(path+'sitzungssummary.pkl','rb')
sitzungen = pickle.load(infile)
infile.close()
sitzungen = sitzungen [['ID','UserID','UserAttribut','Art','HA','beendet','Fehler']]

In [6]:
sitzungen = sitzungen.dropna()
sitzungen.rename(columns = {'ID':'UebungsID'}, inplace = True)

# Userattribut: nur Schüler
options = ['Schüler'] 
sitzungen = sitzungen[sitzungen['UserAttribut'].isin(options)] 

#HA: bereinigung
sitzungen['HA'] = sitzungen['HA'].replace(['frHA'],'Self')

#Art: nur GK
options = ['GK'] 
sitzungen = sitzungen[sitzungen['Art'].isin(options)] 

sitzungen = sitzungen.reset_index()
sitzungen = sitzungen.drop(columns=['UserAttribut','Art','index'])

#HA
dummy_HA = pd.get_dummies(sitzungen['HA'], prefix='HA_')
sitzungen = pd.merge(left=sitzungen, right=dummy_HA,left_index=True,right_index=True)
sitzungen = sitzungen.drop(columns=['HA'])


Merge Sitzungen und Schueler

In [7]:
sitzungen_schueler = pd.merge(left=sitzungen, right=schueler, how='left', on='UserID')
sitzungen_schueler = sitzungen_schueler.dropna()

In [8]:
sitzungen_schueler.to_pickle('sitzungen_schueler_preprocessed.pkl')

Saetze

In [9]:
infile = open(path+'saetze.pkl','rb')
saetze = pickle.load(infile)
infile.close()
saetze = saetze[['satzID','Schwierigkeit','Art','AufgabenID']]

In [10]:
#Art: nur GK
options = ['GK'] 
saetze = saetze[saetze['Art'].isin(options)] 

#Aufgaben ID 0 rausfiltern, weil nur Einsetzübungen betrachtet werden, keine Kompetenztests
options = ['0'] 
saetze = saetze[~saetze['AufgabenID'].isin(options)] 



XMLsaetze

In [11]:
infile = open(path+'xmlsaetze.pkl','rb')
xmlsaetze = pickle.load(infile)
infile.close()
xmlsaetze = xmlsaetze[['ID','UserID','UebungsID','Testposition','SatzID','Erstloesung','Schussel','Datum', 'Erfolg','Loesungsnr']]

infile = open(path+'xmlsaetze_archiv.pkl','rb')
xmlsaetze_archiv = pickle.load(infile)
infile.close()
xmlsaetze_archiv = xmlsaetze_archiv[['ID','UserID','UebungsID','Testposition','SatzID','Erstloesung','Schussel','Datum', 'Erfolg','Loesungsnr']]

xmlsaetze = xmlsaetze.append(xmlsaetze_archiv)

In [12]:
#Merge xmlsaetze mit saetze
xmlsaetze.rename(columns = {'SatzID':'satzID'}, inplace = True)
xmlsaetze= pd.merge(xmlsaetze, saetze, on='satzID', how='inner')

# Create date fields
xmlsaetze['Datum']= pd.to_datetime(xmlsaetze['Datum'])
xmlsaetze['Uhrzeit'] = pd.DatetimeIndex(xmlsaetze['Datum']).hour
xmlsaetze['Wochentag'] = pd.DatetimeIndex(xmlsaetze['Datum']).dayofweek
xmlsaetze['Kalenderwoche'] = pd.DatetimeIndex(xmlsaetze['Datum']).strftime("%V")
xmlsaetze['Monat'] = pd.DatetimeIndex(xmlsaetze['Datum']).month
xmlsaetze['Tag'] = pd.DatetimeIndex(xmlsaetze['Datum']).day

# create ist_schulzeit
def f(row):
    if row['Uhrzeit'] > 14:
        val = 0
    elif row['Uhrzeit'] < 8:
        val = 0
    else:
        val = 1
    return val

xmlsaetze['ist_Schulzeit'] = xmlsaetze.apply(f, axis=1)

# variable mehrfach falsch
xmlsaetze['Loesungsnr']=xmlsaetze['Loesungsnr'].str.split()
xmlsaetze['MehrfachFalsch'] = xmlsaetze['Loesungsnr'].str.len()
xmlsaetze['MehrfachFalsch'] = xmlsaetze['MehrfachFalsch'].replace([np.nan],0)
xmlsaetze['MehrfachFalsch'] = xmlsaetze['MehrfachFalsch'].astype('int')
xmlsaetze['MehrfachFalsch'] = xmlsaetze['MehrfachFalsch'] - 1
xmlsaetze=xmlsaetze.drop(columns='Loesungsnr')

sitzungen_schueler_uebID = sitzungen_schueler.UebungsID
xmlsaetze_GK = xmlsaetze[xmlsaetze['UebungsID'].isin(sitzungen_schueler_uebID)] 

#Testposition
dummy_testposition = pd.get_dummies(xmlsaetze_GK['Testposition'], prefix='Testposition_')
xmlsaetze_GK = pd.merge(left=xmlsaetze_GK, right=dummy_testposition,left_index=True,right_index=True)
xmlsaetze_GK = xmlsaetze_GK.drop(columns=['Testposition'])

xmlsaetze_GK.to_pickle('xmlsaetze_GK_preprocessed.pkl')

In [13]:
sitzungen_schueler = pd.read_pickle('sitzungen_schueler_preprocessed.pkl')
xmlsaetze_GK = pd.read_pickle('xmlsaetze_GK_preprocessed.pkl')


In [14]:
# xmlsaetze und sitzungen mergen on uebungsid / id
final_data =pd.merge(left=xmlsaetze_GK, right=sitzungen_schueler, on='UebungsID', how='left')
final_data = final_data.drop(columns='UserID_y')
final_data.rename(columns = {'UserID_x':'UserID'}, inplace = True)

In [15]:
final_data.to_pickle('final_data_preprocessed.pkl')

In [3]:
df = pd.read_pickle('final_data_preprocessed.pkl')
df.columns

Index(['ID', 'UserID', 'UebungsID', 'satzID', 'Erstloesung', 'Schussel',
       'Datum', 'Erfolg', 'Schwierigkeit', 'Art', 'AufgabenID', 'Uhrzeit',
       'Wochentag', 'Kalenderwoche', 'Monat', 'Tag', 'ist_Schulzeit',
       'MehrfachFalsch', 'Testposition__FT', 'Testposition__nt',
       'Testposition__pruefung', 'Testposition__training',
       'Testposition__version', 'Testposition__vt', 'Testposition__zt',
       'beendet', 'Fehler', 'HA__HA', 'HA__Self', 'HA__nt', 'HA__vt', 'HA__zt',
       'Klassenstufe', 'Jahredabei', 'Sex__m', 'Sex__w'],
      dtype='object')